<a href="https://colab.research.google.com/github/riyachawla-17/FakeNewsDetection/blob/main/fake_news_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

FAKE NEWS DETECTION



In [ ]:
import pandas as pd
import numpy as np
import itertools


In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Mounted at /gdrive
/gdrive


In [ ]:
%cd MyDrive

/gdrive/MyDrive


In [ ]:
%cd Datasets/

/gdrive/MyDrive/Datasets


In [ ]:
df = pd.read_csv("news.csv")

In [ ]:
df.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [ ]:
df.drop(columns = ['title'], inplace=True )

In [ ]:
df

,Unnamed: 0,text,label
0,8476,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,It's primary day in New York and front-runners...,REAL
...,...,...,...
6330,4490,The State Department told the Republican Natio...,REAL
6331,8062,The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...,FAKE
6332,8622,Anti-Trump Protesters Are Tools of the Oligar...,FAKE
6333,4021,"ADDIS ABABA, Ethiopia —President Obama convene...",REAL


In [ ]:
df.shape

(6335, 3)

In [ ]:
df.isnull().sum()

Unnamed: 0    0
text          0
label         0
dtype: int64

In [ ]:
labels = df.label

In [ ]:

labels.head()

0    FAKE
1    FAKE
2    REAL
3    FAKE
4    REAL
Name: label, dtype: object

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
nltk.download('punkt')
nltk.download('stopwords')
ps = PorterStemmer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
import string

In [ ]:
def transform_message(text):
    # change case to lowercase
    message = text.lower()
    # for tokenizing the message
    message = nltk.word_tokenize(text)

    # for removing alpha umeric values
    y = []
    for i in text:
        if i.isalnum():
            y.append(i)

    text = y[:]
    y.clear()

    # for removing stopwords
    for i in text:
        if i not in stopwords.words('english') and i not in string.punctuation:
            y.append(i)

    # for stemming
    text = y[:]
    y.clear()

    for i in text:
        y.append(ps.stem(i))

    return " ".join(y)

In [ ]:
df['text'].apply(transform_message)

0       D n e l G r e e n f e l S h l l n J u r n l F ...
1       G g l e P n e r e D g g L n k e n R e S u b l ...
2       U S S e c r e r f S e J h n F K e r r M n h h ...
3       K e e K n g K e e K n g N v e b e r 9 2 0 1 6 ...
4       I p r r n N e w Y r k n f r n r u n n e r H l ...
                              ...                        
6330    T h e S e D e p r e n l h e R e p u b l c n N ...
6331    T h e P n P B S S h u l S n f r P l u c r c r ...
6332    A n T r u p P r e e r A r e T l f h e O l g r ...
6333    A D D I S A B A B A E h p P r e e n O b c n v ...
6334    J e b B u h I S u e n l A c k n g T r u p H e ...
Name: text, Length: 6335, dtype: object

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(df["text"], labels, test_size = 0.2, random_state = 20)

In [ ]:
x_train.head()

4741    NAIROBI, Kenya — President Obama spoke out Sun...
2089    Killing Obama administration rules, dismantlin...
4074    Dean Obeidallah, a former attorney, is the hos...
5376      WashingtonsBlog \nCNN’s Jake Tapper hit the ...
6028    Some of the biggest issues facing America this...
Name: text, dtype: object

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import PassiveAggressiveClassifier

In [ ]:
vector = TfidfVectorizer(stop_words='english', max_df=0.7)

In [ ]:
tf_train = vector.fit_transform(x_train)
tf_test = vector.transform(x_test)

In [ ]:
pac = PassiveAggressiveClassifier(max_iter=50)
pac.fit(tf_train,y_train)

PassiveAggressiveClassifier(max_iter=50)

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix
y_pred =pac.predict(tf_test)

In [ ]:
score = accuracy_score(y_test,y_pred)

In [ ]:
print(f"Accuracy : {round(score*100,2)}%")

Accuracy : 94.71%


In [ ]:
confusion_matrix(y_test,y_pred,labels=['FAKE','REAL'])

array([[623,  25],
       [ 42, 577]])

In [ ]:
import pickle
filename = 'finalized_model.pkl'
pickle.dump(pac , open(filename, 'wb'))
pickle.dump(vector , open('vectorizer.pkl','wb'))